In [ ]:
import pickle
import datetime
import requests
import time as ti
import pandas as pd
import time
from itertools import count
from dateutil.parser import parse

In [ ]:
error1 = "data/error_2017-01-09.pkl"
e1 = pickle.load(open(error1, 'rb'))

In [ ]:
e1

In [ ]:
file_name = "data/occupancy_2017-01-09.pkl"

In [ ]:
day_dict = pickle.load(open(file_name, 'rb'))

In [ ]:
len(day_dict)

In [ ]:
columns = ['route', 'trip', 'service', 'service_day', 'stop_time', 'capacity2nd', 'capacity1st']#'start', 'products']
df = pd.DataFrame(columns=columns)

ditch_stop =['132','133','134','135','136','137','138', '139', '140', '141','142','174','175', '176']
date="2017-01-09"

for trip in day_dict:
    backend = trip['backend']
    transport = trip['transport']
    #print(backend)
    products = None
    backend_stops = {stop['stop_id']: stop for stop in backend['stops'] if stop['stop_id'] not in ditch_stop}

    for connection in transport['connections']:
        passList = []
        for section in connection['sections']:
            if section['journey']:
                passList = passList + section['journey']['passList']
            
        transport_stops = {stop['station']['id']:stop for stop in passList}
        if backend_stops.keys() == transport_stops.keys():
            break
            
    if not passList:
        raise ValueError()

    service_date = next((d['id'] for d in backend['service']['dates'] if d['date'] == date), None)
    for k, v in backend_stops.items():
        dn = pd.DataFrame(columns=columns, data=[[
                    backend['route']['id'], 
                    backend['id'], 
                    backend['service']['id'],
                    service_date,
                    v['id'],
                    transport_stops[k]['prognosis']['capacity1st'],
                    transport_stops[k]['prognosis']['capacity2nd']
                ]])
        df = pd.concat([df,dn]) 

In [ ]:
df.to_csv('2017-01-09_capacity.csv')

In [ ]:
len(df)

In [ ]:
df.sort_values(['stop_time'], ascending=True)